In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.

In [2]:
excel_path_indata = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/9. Analytical modeling/test_indata.xlsx"
indata_sheet = "test_case_2_whNBD_NBD"
indataDF = pd.read_excel(excel_path_indata,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev
0,1,RDC,Johannesburg,10,40,1,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,3.0
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,2.0
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,4.0
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,4.0
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,5.0
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,6.0
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,6.0
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,3.0
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,3.0


In [3]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))

In [4]:

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "Dealer","Q, subbatches"] = Q_dealer_arr/Q_subbatch_size

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC", "Demand variance"] = sigma2_L * Q_subbatch_size/L_wh

outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,"Q, subbatches",Lead time demand mean,Lead time demand stdev,Demand variance
0,1,RDC,Johannesburg,10,40,1,NaN,NBD,10.0,4.222977,NaN,100.0,42.229773,17.833537
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,3.000000,1.0,NaN,NaN,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,2.000000,1.0,NaN,NaN,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,4.000000,1.0,NaN,NaN,NaN
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,4.000000,1.0,NaN,NaN,NaN
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,5.000000,1.0,NaN,NaN,NaN
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,6.000000,1.0,NaN,NaN,NaN
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,6.000000,1.0,NaN,NaN,NaN
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,3.000000,1.0,NaN,NaN,NaN
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,3.000000,1.0,NaN,NaN,NaN


RDC reorder-point optimization

In [5]:
h_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Holding cost").to_numpy()
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy()
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy()
mu_wh = mu_L/L_wh * Q_subbatch_size

beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))

beta_arr = np.array(beta_list)

beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Shortage cost"] = p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "RDC", "Beta"] = beta_rdc
outdataDF.loc[outdataDF["Type"] == "Dealer", "Beta"] = beta_arr
print(f"Optimal weighted induced backorder cost at the warehouse is: {beta_rdc}, betas are: {beta_arr}")



Optimal weighted induced backorder cost at the warehouse is: 3.349647542084863, betas are: [1.29478612 1.18028801 3.37836326 3.03581267 5.38160667 6.90109501
 6.48272192 2.29390572 2.36760804 1.18028801]


In [6]:
h_rdc = float(indataDF.get(indataDF["Type"] == "RDC").get("Holding cost"))
Q_0 = int(int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))/Q_subbatch_size)

R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)

outdataDF.loc[outdataDF["Type"] == "RDC", "Q, subbatches"] = Q_0
outdataDF.loc[outdataDF["Type"] == "RDC", "R, subbatches"] = R_0
outdataDF


Starting optimizing, R = 0, c = 252.10783982149135, c+1 = 220.05583887461378
Doing upwards optimizing, R = 1, c = 220.05583887461378, c+1 = 189.42450755009168
Doing upwards optimizing, R = 2, c = 189.42450755009168, c+1 = 160.9197594187679
Doing upwards optimizing, R = 3, c = 160.9197594187679, c+1 = 135.26023463984433
Doing upwards optimizing, R = 4, c = 135.26023463984433, c+1 = 113.06607265197209
Doing upwards optimizing, R = 5, c = 113.06607265197209, c+1 = 94.76906524580392
Doing upwards optimizing, R = 6, c = 94.76906524580392, c+1 = 80.56472911003748
Doing upwards optimizing, R = 7, c = 80.56472911003748, c+1 = 70.41057163427652
Doing upwards optimizing, R = 8, c = 70.41057163427652, c+1 = 64.06161345101674
Doing upwards optimizing, R = 9, c = 64.06161345101674, c+1 = 61.12762088615647
Doing upwards optimizing, R = 10, c = 61.12762088615647, c+1 = 61.13623296936731


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,"Q, subbatches",Lead time demand mean,Lead time demand stdev,Demand variance,Shortage cost,Beta,"R, subbatches"
0,1,RDC,Johannesburg,10,40,1,NaN,NBD,10.0,4.222977,4.0,100.0,42.229773,17.833537,NaN,3.349648,10.0
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,3.000000,1.0,NaN,NaN,NaN,19.0,1.294786,NaN
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,2.000000,1.0,NaN,NaN,NaN,19.0,1.180288,NaN
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,4.000000,1.0,NaN,NaN,NaN,19.0,3.378363,NaN
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,4.000000,1.0,NaN,NaN,NaN,19.0,3.035813,NaN
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,5.000000,1.0,NaN,NaN,NaN,19.0,5.381607,NaN
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,6.000000,1.0,NaN,NaN,NaN,19.0,6.901095,NaN
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,6.000000,1.0,NaN,NaN,NaN,19.0,6.482722,NaN
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,3.000000,1.0,NaN,NaN,NaN,19.0,2.293906,NaN
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,3.000000,1.0,NaN,NaN,NaN,19.0,2.367608,NaN


In [7]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] =outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF.loc[outdataDF["Type"] == "RDC", "Lead time"] = indataDF.get(indataDF["Type"] == "RDC").get("Transport time")

In [8]:
opt_dealer_list = []
L_dealer_arr = outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time")
for Q,L_est,fill_rate_target,demand_type,mu,sigma in zip(Q_dealer_arr,L_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr):
    print(demand_type, mu, sigma)
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,mu,demand_variance = math.pow(sigma,2)))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[0])
    fill_rate_dealer_list.append(tup[2])
    exp_stock_on_hand_list.append(tup[3])

outdataDF.loc[outdataDF["Type"] == "Dealer", "R optimal"] = R_opt_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_list

outdataDF.loc[outdataDF["Type"] == "RDC","R optimal"] = R_0*Q_subbatch_size # In units.
outdataDF.loc[outdataDF["Type"] == "RDC","Expected stock on hand"] = positive_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)
outdataDF.loc[outdataDF["Type"] == "RDC","Expected backorders"] = negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)


NBD 1.0 3.0
NBD 1.0 2.0
NBD 1.0 4.0
NBD 1.0 4.0
NBD 1.0 5.0
NBD 1.0 6.0
NBD 1.0 6.0
NBD 1.0 3.0
NBD 1.0 3.0
NBD 1.0 2.0


In [9]:
# List comprehension for exp backorders.
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected backorders"] = [ 
    expected_backorders_discrete(R,Q,lt_mu,exp_stock_on_hand) for 
    R,Q,lt_mu,exp_stock_on_hand in zip(R_opt_dealer_list,Q_dealer_arr,
    outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time demand mean").to_numpy(),
    exp_stock_on_hand_list) ]

In [10]:
# Adding costs
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected holding costs per time unit"] = h_dealer_arr*np.array(exp_stock_on_hand_list)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected shortage costs per time unit"] = outdataDF.get(
    outdataDF["Type"] == "Dealer").get("Expected backorders").to_numpy()*p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Total expected costs"] = outdataDF.get(outdataDF["Type"] == "Dealer").get("Expected holding costs per time unit").to_numpy() + outdataDF.get(outdataDF["Type"] == "Dealer").get("Expected shortage costs per time unit").to_numpy()

outdataDF.loc[outdataDF["Type"] == "RDC", "Expected holding costs per time unit"] = h_rdc*outdataDF.get(outdataDF["Type"] == "RDC").get("Expected stock on hand").to_numpy()
outdataDF.loc[outdataDF["Type"] == "RDC", "Total expected costs"] = outdataDF.get(outdataDF["Type"] == "RDC").get("Expected holding costs per time unit").to_numpy()



In [11]:
excel_path_outdata = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/9. Analytical modeling/test_outdata.xlsx"
outdataDF.to_excel(excel_path_outdata,sheet_name = "Outdata_latest_testrun")

outdataDF


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,...,"R, subbatches",Waiting time,Lead time,R optimal,Realized item fill rate,Expected stock on hand,Expected backorders,Expected holding costs per time unit,Expected shortage costs per time unit,Total expected costs
0,1,RDC,Johannesburg,10,40,1,NaN,NBD,10.0,4.222977,...,10.0,NaN,10.000000,100.0,NaN,33.306010,8.305832,33.306010,NaN,33.306010
1,2,Dealer,Deal1,10,10,1,0.95,NBD,1.0,3.000000,...,NaN,0.830583,10.830583,33.0,0.953738,27.862194,0.192777,27.862194,3.662762,31.524956
2,3,Dealer,Deal2,3,10,1,0.95,NBD,1.0,2.000000,...,NaN,0.830583,3.830583,11.0,0.959886,12.732899,0.063482,12.732899,1.206161,13.939060
3,4,Dealer,Deal3,2,10,1,0.95,NBD,1.0,4.000000,...,NaN,0.830583,2.830583,30.0,0.950730,32.776692,0.107276,32.776692,2.038237,34.814930
4,5,Dealer,Deal4,1,10,1,0.95,NBD,1.0,4.000000,...,NaN,0.830583,1.830583,28.0,0.952121,31.742511,0.073094,31.742511,1.388787,33.131298
5,6,Dealer,Deal5,4,10,1,0.95,NBD,1.0,5.000000,...,NaN,0.830583,4.830583,51.0,0.951044,51.848269,0.178853,51.848269,3.398198,55.246468
6,7,Dealer,Deal6,3,10,1,0.95,NBD,1.0,6.000000,...,NaN,0.830583,3.830583,68.0,0.950133,69.831196,0.161779,69.831196,3.073797,72.904992
7,8,Dealer,Deal7,2,10,1,0.95,NBD,1.0,6.000000,...,NaN,0.830583,2.830583,66.0,0.950907,68.791845,0.122428,68.791845,2.326127,71.117972
8,9,Dealer,Deal8,5,10,1,0.95,NBD,1.0,3.000000,...,NaN,0.830583,5.830583,24.0,0.953858,23.805238,0.135822,23.805238,2.580609,26.385848
9,10,Dealer,Deal9,4,10,1,0.95,NBD,1.0,3.000000,...,NaN,0.830583,4.830583,22.0,0.953589,22.791425,0.122008,22.791425,2.318160,25.109585


In [12]:
print(f"Length: {len(rdc_f_u_probability_array)}, p(0) = {rdc_f_u_probability_array[0]}")

Length: 41, p(0) = 0.0006205779022028941


In [13]:
rdc_f_u_probability_array

array([6.20577902e-04, 3.47983637e-03, 1.05207135e-02, 2.27455374e-02,
       3.93792854e-02, 5.80013366e-02, 7.54375661e-02, 8.88327458e-02,
       9.64082223e-02, 9.77095461e-02, 9.34175690e-02, 8.49250627e-02,
       7.38795586e-02, 6.18230822e-02, 4.99785578e-02, 3.91733101e-02,
       2.98605746e-02, 2.21944033e-02, 1.61215351e-02, 1.14667133e-02,
       7.99994094e-03, 5.48284995e-03, 3.69639939e-03, 2.45426533e-03,
       1.60655836e-03, 1.03781243e-03, 6.62159500e-04, 4.17605353e-04,
       2.60517112e-04, 1.60861746e-04, 9.83718694e-05, 5.96107341e-05,
       3.58119560e-05, 2.13392509e-05, 1.26171151e-05, 7.40524071e-06,
       4.31591178e-06, 2.49864781e-06, 1.43737910e-06, 8.21858159e-07,
       4.67195613e-07])